In [1]:
# %load init.ipy
%reload_ext autoreload
%autoreload 2

from importlib import reload

# import os
# import sys
# import logging
import warnings
# import glob
# from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# import scipy as sp
# import scipy.interpolate
# import astropy as ap
# import h5py


# --- LZK Libs ---

import zcode.math as zmath
import zcode.plot as zplot
# import zcode.astro as zastro
from zcode.constants import *
import zcode.inout as zio

# import kalepy as kale


# --- Arepo / Illustris ---

import illpy
import illpy_lib
import illpy_lib.illcosmo
import illpy_lib.illbh
# import illpy_lib.illbh.mergers
# import illpy_lib.illbh.details
# import illpy_lib.illbh.snapshots
# import illpy_lib.illbh.groupcats

from illpy_lib.illbh import BH_TYPE


# --- Generic Setup ---

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore');
%precision %.2g

warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 12})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
plt.rcParams.update({'grid.alpha': 0.5})


# --- Project Setup ---

# PATH_BASE = "/blue/lblecha/lkelley/arepo_sims/cosmo_zooms/dynamics-comparison"
# PATH_BASE = "/Volumes/Smally/arepo_sims/"

# SIM_NAME = "bh_new_centering"

# sim_path = os.path.join(PATH_BASE, SIM_NAME, 'output', '')

# PATH = (
# #     "/Volumes/Smally/arepo_sims/"
# #     "bh_new_centering/output"
#     "new-log_pos-test/output"
# #     "/Volumes/Smally/log_test_data/"
# )

# SIM_PATHS = [
#     "bh_new_centering",
#     "repos-potmin",
#     "unalt-dyn"
# ]

print("done")

done


In [2]:
SIM_PATH = (
    "/net/hernquistfs3/hernquistfs3/share_root/IllustrisTNG/Runs/"
    "L75n1820TNG/"
)
PROC_PATH = (
    "/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/illustris-data/newer/"
    "L75n1820TNG/"
)

In [3]:
import illpy_lib.illbh.snapshots
reload(illpy_lib.illbh.snapshots)
snap_99 = illpy_lib.illbh.snapshots.Snapshots_Snap(99, SIM_PATH, PROC_PATH, recreate=True)
# snaps = illpy_lib.illbh.snapshots.Snapshots(SIM_PATH, PROC_PATH, recreate=False, verbose=True)

Running `_process()`; recreate: True, exists: True (/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/illustris-data/newer/L75n1820TNG/bh-snapshots/bh-snapshots_099.hdf5)
Loaded header time=1.00000000, nbh=37053
Loaded 37053 BHs from snapshot 99
Saved data to '/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/illustris-data/newer/L75n1820TNG/bh-snapshots/bh-snapshots_099.hdf5' size 5.0 MB
Loaded      37053 entries from '/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/illustris-data/newer/L75n1820TNG/bh-snapshots/bh-snapshots_099.hdf5', created '2020-11-01 17:47:43.304757'
Running `_process()`; recreate: True, exists: True (/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/illustris-data/newer/L75n1820TNG/bh-snapshots/bh-snapshots_099.hdf5)
Loaded header time=1.00000000, nbh=37053
Loaded 37053 BHs from snapshot 99
Saved data to '/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/illustris-data/newer/L75n1820TNG/bh-snapshots/bh-snapshots_099.hdf5' size

In [ ]:
snaps.keys()

In [4]:
import illpy_lib.illbh.mergers
reload(illpy_lib.illbh.mergers)
mergers = illpy_lib.illbh.mergers.Mergers_TNG(SIM_PATH, PROC_PATH, recreate=False, verbose=True)

Loaded      30973 entries from '/n/holystore01/LABS/hernquist_lab/Lab/ghernquist/lkelley/illustris-data/newer/L75n1820TNG/bh-mergers.hdf5', created '2020-10-18 18:18:27.985858'


In [5]:
mergers.keys()

['BH_Mass',
 'Masses',
 'ParticleIDs',
 'scale',
 'task',
 'unique_counts',
 'unique_ids',
 'unique_indices']

In [22]:
reload(illpy_lib.illbh)
import illpy_lib.illbh.details
reload(illpy_lib.illbh.details)
dets_t0 = illpy_lib.illbh.details.Details_TNG_Task(0, SIM_PATH, PROC_PATH, recreate=False, verbose=True)

Found parameters file: '/net/hernquistfs3/hernquistfs3/share_root/IllustrisTNG/Runs/L75n1820TNG/arepo/param.txt-usedvalues'
Found 100 snapshot directories
Snap 0: loading header info from '/net/hernquistfs3/hernquistfs3/share_root/IllustrisTNG/Runs/L75n1820TNG/output/snapdir_000/snap_000.144.hdf5'
Files exist


In [8]:
import datetime
import illpy_lib.illbh.details
beg = datetime.datetime.now()

task = 0
cosmo = None
while task < 1e6:
    now = datetime.datetime.now()
    print("\n", task, "at {} after {}".format(now, now - beg))
    try:
        temp = illpy_lib.illbh.details.Details_TNG_Task(
            task, SIM_PATH, PROC_PATH,
            cosmo=cosmo, recreate=False, verbose=True)
        cosmo = temp._cosmo
    except FileNotFoundError as err:
        print(f"Completed with {task} tasks")
        break
        
    task += 1
    # break
    
now = datetime.datetime.now()
print("\ndone at {} after {}".format(now, now - beg))


 0 at 2020-11-01 21:08:24.303126 after 0:00:00.000141
Found parameters file: '/net/hernquistfs3/hernquistfs3/share_root/IllustrisTNG/Runs/L75n1820TNG/arepo/param.txt-usedvalues'
Found 100 snapshot directories
Snap 0: loading header info from '/net/hernquistfs3/hernquistfs3/share_root/IllustrisTNG/Runs/L75n1820TNG/output/snapdir_000/snap_000.144.hdf5'
Files exist

 1 at 2020-11-01 21:08:55.744303 after 0:00:31.441318
Files exist

 2 at 2020-11-01 21:08:55.789590 after 0:00:31.486605
Files exist

 3 at 2020-11-01 21:08:55.819159 after 0:00:31.516174
Files exist

 4 at 2020-11-01 21:08:55.853055 after 0:00:31.550070
Files exist

 5 at 2020-11-01 21:08:55.885862 after 0:00:31.582877
Files exist

 6 at 2020-11-01 21:08:55.930427 after 0:00:31.627442
Files exist

 7 at 2020-11-01 21:08:55.966546 after 0:00:31.663561
Files exist

 8 at 2020-11-01 21:08:55.996530 after 0:00:31.693545
Files exist

 9 at 2020-11-01 21:08:56.073710 after 0:00:31.770725
Files exist

 10 at 2020-11-01 21:08:56.164

Files exist

 121 at 2020-11-01 21:09:03.087310 after 0:00:38.784325
Files exist

 122 at 2020-11-01 21:09:03.141583 after 0:00:38.838598
Files exist

 123 at 2020-11-01 21:09:03.196488 after 0:00:38.893503
Files exist

 124 at 2020-11-01 21:09:03.226240 after 0:00:38.923255
Files exist

 125 at 2020-11-01 21:09:03.282669 after 0:00:38.979684
Files exist

 126 at 2020-11-01 21:09:03.315444 after 0:00:39.012459
Files exist

 127 at 2020-11-01 21:09:03.346207 after 0:00:39.043222
Files exist

 128 at 2020-11-01 21:09:03.403122 after 0:00:39.100137
Files exist

 129 at 2020-11-01 21:09:03.433767 after 0:00:39.130782
Files exist

 130 at 2020-11-01 21:09:03.490597 after 0:00:39.187612
Files exist

 131 at 2020-11-01 21:09:03.552457 after 0:00:39.249472
Files exist

 132 at 2020-11-01 21:09:03.610377 after 0:00:39.307392
Files exist

 133 at 2020-11-01 21:09:03.667189 after 0:00:39.364204
Files exist

 134 at 2020-11-01 21:09:03.767904 after 0:00:39.464919
Files exist

 135 at 2020-11-01 21

Files exist

 243 at 2020-11-01 21:09:10.574842 after 0:00:46.271857
Files exist

 244 at 2020-11-01 21:09:10.605978 after 0:00:46.302993
Files exist

 245 at 2020-11-01 21:09:10.662359 after 0:00:46.359374
Files exist

 246 at 2020-11-01 21:09:10.754571 after 0:00:46.451586
Files exist

 247 at 2020-11-01 21:09:10.835405 after 0:00:46.532420
Files exist

 248 at 2020-11-01 21:09:10.869503 after 0:00:46.566518
Files exist

 249 at 2020-11-01 21:09:10.945197 after 0:00:46.642212
Files exist

 250 at 2020-11-01 21:09:11.023259 after 0:00:46.720274
Files exist

 251 at 2020-11-01 21:09:11.118805 after 0:00:46.815820
Files exist

 252 at 2020-11-01 21:09:11.175871 after 0:00:46.872886
Files exist

 253 at 2020-11-01 21:09:11.234722 after 0:00:46.931737
Files exist

 254 at 2020-11-01 21:09:11.267039 after 0:00:46.964054
Files exist

 255 at 2020-11-01 21:09:11.330787 after 0:00:47.027802
Files exist

 256 at 2020-11-01 21:09:11.391096 after 0:00:47.088111
Files exist

 257 at 2020-11-01 21

Files exist

 364 at 2020-11-01 21:09:17.743930 after 0:00:53.440945
Files exist

 365 at 2020-11-01 21:09:17.801214 after 0:00:53.498229
Files exist

 366 at 2020-11-01 21:09:17.861790 after 0:00:53.558805
Files exist

 367 at 2020-11-01 21:09:17.922065 after 0:00:53.619080
Files exist

 368 at 2020-11-01 21:09:18.043346 after 0:00:53.740361
Files exist

 369 at 2020-11-01 21:09:18.136918 after 0:00:53.833933
Files exist

 370 at 2020-11-01 21:09:18.195717 after 0:00:53.892732
Files exist

 371 at 2020-11-01 21:09:18.228028 after 0:00:53.925043
Files exist

 372 at 2020-11-01 21:09:18.265461 after 0:00:53.962476
Files exist

 373 at 2020-11-01 21:09:18.324685 after 0:00:54.021700
Files exist

 374 at 2020-11-01 21:09:18.357851 after 0:00:54.054866
Files exist

 375 at 2020-11-01 21:09:18.415160 after 0:00:54.112175
Files exist

 376 at 2020-11-01 21:09:18.448191 after 0:00:54.145206
Files exist

 377 at 2020-11-01 21:09:18.510285 after 0:00:54.207300
Files exist

 378 at 2020-11-01 21

Files exist

 486 at 2020-11-01 21:09:25.276565 after 0:01:00.973580
Files exist

 487 at 2020-11-01 21:09:25.334981 after 0:01:01.031996
Files exist

 488 at 2020-11-01 21:09:25.393604 after 0:01:01.090619
Files exist

 489 at 2020-11-01 21:09:25.452917 after 0:01:01.149932
Files exist

 490 at 2020-11-01 21:09:25.488782 after 0:01:01.185797
Files exist

 491 at 2020-11-01 21:09:25.561773 after 0:01:01.258788
Files exist

 492 at 2020-11-01 21:09:25.621204 after 0:01:01.318219
Files exist

 493 at 2020-11-01 21:09:25.655628 after 0:01:01.352643
Files exist

 494 at 2020-11-01 21:09:25.720333 after 0:01:01.417348
Files exist

 495 at 2020-11-01 21:09:25.796144 after 0:01:01.493159
Files exist

 496 at 2020-11-01 21:09:25.866905 after 0:01:01.563920
Files exist

 497 at 2020-11-01 21:09:25.898714 after 0:01:01.595729
Files exist

 498 at 2020-11-01 21:09:25.959878 after 0:01:01.656893
Files exist

 499 at 2020-11-01 21:09:26.019095 after 0:01:01.716110
Files exist

 500 at 2020-11-01 21

...

Files exist

 10426 at 2020-11-01 21:17:56.758799 after 0:09:32.455814
Files exist

 10427 at 2020-11-01 21:17:56.815928 after 0:09:32.512943
Files exist

 10428 at 2020-11-01 21:17:56.848635 after 0:09:32.545650
Files exist

 10429 at 2020-11-01 21:17:56.908133 after 0:09:32.605148
Files exist

 10430 at 2020-11-01 21:17:56.943611 after 0:09:32.640626
Files exist

 10431 at 2020-11-01 21:17:57.020200 after 0:09:32.717215
Files exist

 10432 at 2020-11-01 21:17:57.081874 after 0:09:32.778889
Files exist

 10433 at 2020-11-01 21:17:57.140133 after 0:09:32.837148
Files exist

 10434 at 2020-11-01 21:17:57.199587 after 0:09:32.896602
Files exist

 10435 at 2020-11-01 21:17:57.268670 after 0:09:32.965685
Files exist

 10436 at 2020-11-01 21:17:57.329315 after 0:09:33.026330
Files exist

 10437 at 2020-11-01 21:17:57.364885 after 0:09:33.061900
Files exist

 10438 at 2020-11-01 21:17:57.400975 after 0:09:33.097990
Files exist

 10439 at 2020-11-01 21:17:57.464138 after 0:09:33.161153
Files 

Files exist

 10543 at 2020-11-01 21:18:02.828874 after 0:09:38.525889
Files exist

 10544 at 2020-11-01 21:18:02.887383 after 0:09:38.584398
Files exist

 10545 at 2020-11-01 21:18:02.918058 after 0:09:38.615073
Files exist

 10546 at 2020-11-01 21:18:02.979338 after 0:09:38.676353
Files exist

 10547 at 2020-11-01 21:18:03.039659 after 0:09:38.736674
Files exist

 10548 at 2020-11-01 21:18:03.071339 after 0:09:38.768354
Files exist

 10549 at 2020-11-01 21:18:03.104096 after 0:09:38.801111
Files exist

 10550 at 2020-11-01 21:18:03.161800 after 0:09:38.858815
Files exist

 10551 at 2020-11-01 21:18:03.193527 after 0:09:38.890542
Files exist

 10552 at 2020-11-01 21:18:03.254201 after 0:09:38.951216
Files exist

 10553 at 2020-11-01 21:18:03.316366 after 0:09:39.013381
Files exist

 10554 at 2020-11-01 21:18:03.346836 after 0:09:39.043851
Files exist

 10555 at 2020-11-01 21:18:03.379966 after 0:09:39.076981
Files exist

 10556 at 2020-11-01 21:18:03.440692 after 0:09:39.137707
Files 

Files exist

 10660 at 2020-11-01 21:18:08.564130 after 0:09:44.261145
Files exist

 10661 at 2020-11-01 21:18:08.623473 after 0:09:44.320488
Files exist

 10662 at 2020-11-01 21:18:08.657257 after 0:09:44.354272
Files exist

 10663 at 2020-11-01 21:18:08.716862 after 0:09:44.413877
Files exist

 10664 at 2020-11-01 21:18:08.785425 after 0:09:44.482440
Files exist

 10665 at 2020-11-01 21:18:08.819846 after 0:09:44.516861
Files exist

 10666 at 2020-11-01 21:18:08.879883 after 0:09:44.576898
Files exist

 10667 at 2020-11-01 21:18:08.944196 after 0:09:44.641211
Files exist

 10668 at 2020-11-01 21:18:09.008252 after 0:09:44.705267
Files exist

 10669 at 2020-11-01 21:18:09.067486 after 0:09:44.764501
Files exist

 10670 at 2020-11-01 21:18:09.103487 after 0:09:44.800502
Files exist

 10671 at 2020-11-01 21:18:09.163926 after 0:09:44.860941
Files exist

 10672 at 2020-11-01 21:18:09.224542 after 0:09:44.921557
Files exist

 10673 at 2020-11-01 21:18:09.283867 after 0:09:44.980882
Files 

ERROR:root:ERROR: `fname` '/net/hernquistfs3/hernquistfs3/share_root/IllustrisTNG/Runs/L75n1820TNG/output/blackhole_details/blackhole_details_10752.txt' does not exist!


Completed with 10752 tasks

done at 2020-11-01 21:18:13.636819 after 0:09:49.333834


In [ ]:
illpy_lib.illbh.details.process_details_snaps__tng(
    SIM_PATH, PROC_PATH, recreate=True, verbose=True)

In [ ]:
dets = illpy_lib.illbh.details.Details_TNG(
    SIM_PATH, processed_path=PROC_PATH, recreate=False, verbose=True)

In [ ]:
snaps = illpy_lib.illbh.snapshots.Snapshots_TNG(SIM_PATH, processed_path=PROC_PATH, recreate=True)